# Building ANCOR Dataset

In [ ]:
from french_crs.ancor2dataset import dataset_builder

# Do-able with two nested loops
# for method in ["balanced", "representative", "window"]:
#     for subcorpus in [["INDIRECTE"],["DIRECTE"],["ANAPHORE"]]:

dataset_params={
    # There are other three possible strategies: (1)"balanced", (2)"representative" and (3)"window"
    "strategy" : "balanced,
    "ancor_corpus_path" : "../DISTRIB_ANCOR/",
    # All possibilities are: ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"]
    "sub_corpus_filter" : ["corpus_OTG","corpus_UBS","corpus_ESLO" ,"corpus_ESLO_CO2"], 
    # All possibilities are: ["DIRECTE", "INDIRECTE", "ANAPHORE"]
    "coreference_type_filter" : ["DIRECTE"], 
    "dataset_output_folder" : "../datasets/",
    # This will be used only in window strategy
    "window_size" : 5 
}

dataset = dataset_builder(**dataset_params)
dataset.build_dataset(file_analysis_alert=True)
dataset.merge_dataset(delete_original_after_merge=False)

In [45]:
import glob

json_relations_lined_up={}
counter=0

for sub_corpus in ["corpus_OTG","corpus_UBS","corpus_ESLO_CO2","corpus_ESLO"]:

    ancor_subcorpus_path = "../DISTRIB_ANCOR_EN_CHAINE_Medium/"+sub_corpus+'/'

    corpus_files = [file[file.rfind(
        "/")+1:-3] for file in glob.glob(ancor_subcorpus_path+"aa_fichiers/*.aa")]

    list_dataframes = []
    
    for subfile in corpus_files:
        
        print("starting",subfile,"at",sub_corpus)

        root_xml,root_aa,data_source=set_file_config("../DISTRIB_ANCOR_EN_CHAINE_Medium/",sub_corpus, subfile)
        print("starting 1")

        json_mentions=generate_json_mentions(root_xml,root_aa)
        print("starting 2")

        json_mentions=remove_empty_json_mentions(json_mentions)
        print("starting 3")

        json_relations=generate_json_relations()
        print("starting 4")

#         json_text=generate_json_text_with_mentions()
        if json_relations == {}:
            continue
        json_coreference_chains=generate_json_chains()
        print("starting 5")

        
        
        
        for pair_id in json_relations:            

            if json_relations[pair_id]["TYPE"] in ["DIRECTE", "INDIRECTE", "ANAPHORE"]:
                
                counter+=1
                coref_features=generate_coref_relations_json_features(pair_id,counter,json_relations,
                                                                      json_mentions,data_source,
                                                                      is_positive_instance=True)
                json_relations_lined_up[counter]=coref_features

            
                list_mentions_sorted_seperated=[]
                df_mentions = pd.DataFrame.from_dict(json_mentions, orient='index')
                df_mentions = df_mentions.sort_values(by=['START_ID'])
                list_mentions_sorted = list(df_mentions.index)
                
                relation_left=json_relations[pair_id]["LEFT_UNIT"]["ID"]
                relation_right=json_relations[pair_id]["RIGHT_UNIT"]["ID"]
                
                if json_mentions[relation_left]["START_ID"] > json_mentions[relation_right]["START_ID"]:
                    relation_left,relation_right=relation_right,relation_left

                start_index=list_mentions_sorted.index(relation_left)+1
                end_index=list_mentions_sorted.index(relation_right)

                mentions_in_between=list_mentions_sorted[start_index:end_index]

                result="Nothing"
                for ment in reversed(mentions_in_between):
                    for index, chain in json_coreference_chains.items():
                        if not (ment in json_coreference_chains[index] and  relation_right in json_coreference_chains[index]): 
                                result=(ment,relation_right)
                                break
                    else:
                        continue

                    break
                    
                if result!="Nothing":
                    
                    counter+=1
                    coref_features=generate_coref_relations_json_features(-1,counter,json_relations,
                                                                          json_mentions,data_source,
                                                                          is_positive_instance=False,
                                                                          pair_left_id=result[0],
                                                                          pair_right_id=result[1])
                    json_relations_lined_up[counter]=coref_features
                    
                    

            if json_relations[pair_id]["TYPE"] in ["ASSOC", "ASSOC_PRONOM"]:

                counter+=1
                coref_features=generate_coref_relations_json_features(pair_id,counter,json_relations,
                                                                      json_mentions,data_source,
                                                                      is_positive_instance=True,
                                                                      is_assoc=True)
                json_relations_lined_up[counter]=coref_features


df_coref_relations = pd.DataFrame.from_dict(json_relations_lined_up, orient='index')
df_coref_relations.to_excel("Dataset_ANCOR_XXXXX.xlsx")

df_coref_relations

starting 2AG0619 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 2SB0008 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1SB0302 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0438 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0513 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1SB0213 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 1SB0031 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0620 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 2AP0259 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0527 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0576 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0585 at corpus_OTG
starting 1
starting 2
starting 3
s

starting 2
starting 3
starting 4
starting 5
starting 2SB0315 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0391 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 2AG0561 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 1PF0358 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0362 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0416 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 2AP0092 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0447 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0509 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 1AG0558 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 2AP0144 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AP0596 at corpus_OTG
starting 1
s

starting 1PF0444 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 2AP0180 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 2AP0287 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1SB0028 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 2AP0225 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0421 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0470 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0450 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 2AP0205 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AG0499 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0464 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0646 at corpus_OTG
starting 1
starting 2
s

starting 2
starting 3
starting 4
starting 5
starting 1SB0043 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0639 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AP0061 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AP0170 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AP0243 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0052 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AP0115 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1PF0392 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 1AP0226 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 1AP0366 at corpus_OTG
starting 1
starting 2
starting 3
starting 4
starting 5
starting 054_00000037 at corpus_UBS
starting 1
starting 2
starting 3
starting 4
starting 5
starting 073_000

,SUB_CORPUS,FILE_NAME,COREF_TABLE_ID,COREF_TYPE,COREF_ANCOR_ID,MENTION_LEFT_ID,MENTION_RIGHT_ID,LEFT_CONTENT,RIGHT_CONTENT,LEFT_TYPE,...,ID_EN,ID_GENDER,ID_NUMBER,DISTANCE_MENTION,DISTANCE_WORD,DISTANCE_CHAR,EMBEDDED,ID_PREVIOUS,ID_NEXT,IS_CO_REF
1,corpus_OTG,2AG0619,1,DIRECTE,adrouin_1331117659587,adrouin_1329170635685,adrouin_1329170699645,Grenoble,Grenoble,N,...,YES,UNK,NO,0,0,0,YES,NO,NO,1
2,corpus_OTG,2AG0619,2,DIRECTE,adrouin_1331117663612,adrouin_1329170584704,adrouin_1329170747865,Lyon,Lyon,N,...,YES,UNK,NO,4,10,31,YES,YES,NO,1
3,corpus_OTG,2AG0619,3,NON_COREFERENCE,-1,adrouin_1329170904629,adrouin_1329170747865,on,Lyon,PR,...,NO,UNK,NO,0,3,7,YES,NO,NO,0
4,corpus_OTG,2AG0619,4,DIRECTE,adrouin_1331117667106,adrouin_1329170699645,adrouin_1329170838501,Grenoble,Grenoble,N,...,YES,UNK,NO,3,11,27,YES,NO,YES,1
5,corpus_OTG,2AG0619,5,NON_COREFERENCE,-1,adrouin_1329170747865,adrouin_1329170838501,Lyon,Grenoble,N,...,YES,UNK,NO,0,3,10,NO,YES,NO,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12637,corpus_ESLO,004_-1,12637,INDIRECTE,jmuzerelle_1371569939851,jmuzerelle_1353935608198,jmuzerelle_1353935675332,la moitié de sa bourse,le chiffre,N,...,YES,NO,NO,3,4,18,NO,UNK,NO,1
12638,corpus_ESLO,004_-1,12638,NON_COREFERENCE,-1,jmuzerelle_1353935890125,jmuzerelle_1353935675332,on,le chiffre,PR,...,NO,YES,NO,0,1,7,NO,UNK,NO,0
12639,corpus_ESLO,004_-1,12639,ANAPHORE,jmuzerelle_1371571005103,jmuzerelle_1353680305415,jmuzerelle_1353680332653,en,en,PR,...,YES,YES,NO,1,8,42,YES,YES,YES,1
12640,corpus_ESLO,004_-1,12640,NON_COREFERENCE,-1,jmuzerelle_1353680327848,jmuzerelle_1353680332653,je,en,PR,...,NO,YES,NO,0,4,18,NO,UNK,NO,0


In [23]:
def generate_coref_relations_json_features(pair_id,counter_id,json_relations,json_mentions,
                                           data_source,is_positive_instance=True,
                                           pair_left_id=None,pair_right_id=None,is_assoc=False):

    if is_positive_instance:
        relation_type=json_relations[pair_id]["TYPE"]
        relation_left=json_relations[pair_id]["LEFT_UNIT"]
        relation_right=json_relations[pair_id]["RIGHT_UNIT"]
        
        if relation_left["START_ID"] > relation_right["START_ID"]:
            relation_left,relation_right=relation_right,relation_left
        
        MENTION_LEFT_ID=relation_left["ID"]
        MENTION_RIGHT_ID=relation_right["ID"]
        
        if is_assoc:
            is_positive_instance=False
            
        
    else:
        relation_type="NON_COREFERENCE"
        relation_left=json_mentions[pair_left_id]
        relation_right=json_mentions[pair_right_id]
        
        if relation_left["START_ID"] > relation_right["START_ID"]:
            relation_left,relation_right=relation_right,relation_left
        
        MENTION_LEFT_ID=pair_left_id
        MENTION_RIGHT_ID=pair_right_id




    string_text = data_source[relation_left["END_ID"]:relation_right["START_ID"]]
    string_text=re.sub("<[a-zA-Z0-9\s=\",/.]+>", " ", string_text).strip().split()
    DISTANCE_MENTION = int(relation_right["NUM"])-int(relation_left["NUM"])-1
    DISTANCE_WORD = len(string_text)
    DISTANCE_CHAR = len("".join(string_text))


    tf2yn = {True: "YES", False: "NO"}

    left_content = str(relation_left["CONTENT"])
    right_content = str(relation_right["CONTENT"])
    left_content_list = left_content.lower().split()
    right_content_list = right_content.lower().split()
    len_of_words_min = min(len(left_content_list), len(right_content_list))
    len_of_words_max = max(len(left_content_list), len(right_content_list))
    len_intersection = len(
        [value for value in left_content_list if value in right_content_list])


    ID_FORM = tf2yn[relation_left["CONTENT"]==relation_right["CONTENT"]]
    ID_SUBFORM = tf2yn[len_intersection>0]

    INCL_RATE = len_intersection/len_of_words_min
    COM_RATE = len_intersection/len_of_words_max

    ID_DEF = tf2yn[relation_left["DEF"]==relation_right["DEF"]]
    if relation_left["DEF"] == "UNK" or relation_right["DEF"] == "UNK":
        ID_DEF = "UNK"

    ID_GP = tf2yn[relation_left["GP"]==relation_right["GP"]]
    if relation_left["GP"] == "UNK" or relation_right["GP"] == "UNK":
        ID_GP = "UNK"

    ID_TYPE = tf2yn[relation_left["TYPE"]==relation_right["TYPE"]]
    if relation_left["TYPE"] == "UNK" or relation_right["TYPE"] == "UNK":
        ID_TYPE = "UNK"

    ID_EN = tf2yn[relation_left["EN"]==relation_right["EN"]]
    if relation_left["EN"] == "UNK" or relation_right["EN"] == "UNK":
        ID_EN = "UNK"

    ID_GENDER = tf2yn[relation_left["GENRE"]==relation_right["GENRE"]]
    if relation_left["GENRE"] == "UNK" or relation_right["GENRE"] == "UNK":
        ID_GENDER = "UNK"

    ID_NUMBER = tf2yn[relation_left["NUM"]==relation_right["NUM"]]
    if relation_left["NUM"] == "UNK" or relation_right["NUM"] == "UNK":
        ID_NUMBER = "UNK"

    EMBEDDED = tf2yn[((left_content in right_content) or (right_content in left_content))]

    ID_PREVIOUS = tf2yn[relation_left["PREVIOUS"]==relation_right["PREVIOUS"]]
    if relation_left["PREVIOUS"] == "^" or relation_right["PREVIOUS"] == "^":
        ID_PREVIOUS = "UNK"

    ID_NEXT = tf2yn[relation_left["NEXT"]==relation_right["NEXT"]]
    if relation_left["NEXT"] == "^" or relation_right["NEXT"] == "^":
        ID_NEXT = "UNK"
        
    coref_relation_json_feature={
        
                                "SUB_CORPUS":sub_corpus,
                                "FILE_NAME":subfile,
                                "COREF_TABLE_ID":counter_id,
                                "COREF_TYPE":relation_type,
                                "COREF_ANCOR_ID":pair_id,
                                "MENTION_LEFT_ID":MENTION_LEFT_ID,
                                "MENTION_RIGHT_ID":MENTION_RIGHT_ID,
                                "LEFT_CONTENT":relation_left["CONTENT"],                                     
                                "RIGHT_CONTENT":relation_right["CONTENT"],                                     
                                "LEFT_TYPE":relation_left["TYPE"], 
                                "RIGHT_TYPE":relation_right["TYPE"], 
                                "LEFT_DEF":relation_left["DEF"],
                                "RIGHT_DEF":relation_right["DEF"], 
                                "LEFT_GP":relation_left["GP"],
                                "RIGHT_GP":relation_right["GP"],
                                "LEFT_GENRE":relation_left["GENRE"],
                                "RIGHT_GENRE":relation_right["GENRE"],
                                "LEFT_NB":relation_left["NB"],
                                "RIGHT_NB":relation_right["NB"],
                                "LEFT_EN":relation_left["EN"],
                                "RIGHT_EN":relation_right["EN"],
                                "ID_FORM": ID_FORM, 
                                "ID_SUBFORM": ID_SUBFORM, 
                                "INCL_RATE": INCL_RATE, 
                                "COM_RATE": COM_RATE, 
                                "ID_DEF": ID_DEF, 
                                "ID_GP": ID_GP, 
                                "ID_TYPE": ID_TYPE,
                                "ID_EN": ID_EN, 
                                "ID_GENDER": ID_GENDER, 
                                "ID_NUMBER": ID_NUMBER, 
                                "DISTANCE_MENTION": DISTANCE_MENTION, 
                                "DISTANCE_WORD": DISTANCE_WORD,
                                "DISTANCE_CHAR": DISTANCE_CHAR, 
                                "EMBEDDED": EMBEDDED, 
                                "ID_PREVIOUS": ID_PREVIOUS, 
                                "ID_NEXT": ID_NEXT, 
                                "IS_CO_REF": int(is_positive_instance)
    }
    
    return(coref_relation_json_feature)
                    

In [24]:
import xml.etree.ElementTree as ET
import json
import pandas as pd
import re
import random
import glob
import networkx as nx
import os


def set_file_config(path_to_ancor, sub_corpus, file):

    path_file = path_to_ancor+sub_corpus+'/'

    tree_xml = ET.parse(
        path_file+'annotation_integree/'+file+'.xml')
    tree_aa = ET.parse(
        path_file+'aa_fichiers/'+file+'.aa')
    data_source = open(
        path_file+'ac_fichiers/'+file + '.ac', "r").read()

    root_xml = tree_xml.getroot()
    root_aa = tree_aa.getroot()
    
    return(root_xml,root_aa,data_source)
    

# return mentions in json format
def generate_json_mentions(root_xml,root_aa):

    json_mentions = {}

    for unit in root_xml.iter("unit"):
        data_character = {}
        data_character["TYPE"] = unit.find("./characterisation/type").text

        for anchor in root_xml.iter("anchor"):
            if anchor.attrib["id"] == unit.attrib["id"]:
                data_character["NUM"] = anchor.attrib["num"]

        for feat in unit.findall("./characterisation/featureSet/feature"):
            data_character[feat.attrib["name"]] = feat.text

        for unit_aa in root_aa.iter("unit"):
            if unit_aa.attrib["id"] == unit.attrib["id"]:
                data_character["START_ID"] = int(unit_aa.find(
                    "./positioning/start/singlePosition").attrib["index"])
                data_character["END_ID"] = int(unit_aa.find(
                    "./positioning/end/singlePosition").attrib["index"])

        json_mentions[unit.attrib["id"]] = data_character

    return(json_mentions)

# return mentions in json format and eliminates all the features that are not presented.
def remove_empty_json_mentions(json_mentions,features_to_check=["CONTENT", "PREVIOUS"]):

    json_mentions_= json_mentions.copy()
    for mention in list(json_mentions_):
        for feature in features_to_check:
            if feature not in json_mentions_[mention]:
                del json_mentions_[mention]
                break

    return(json_mentions_)


# return relations in json format based on the json mentions
def generate_json_relations():

    json_relations={}

    for relation in root_aa.iter("relation"):
        data_character = {}
        data_character["TYPE"] = relation.find("./characterisation/type").text

        for feat in relation.findall("./characterisation/featureSet/feature"):
            data_character[feat.attrib["name"]] = feat.text

        unit_ids = []
        for feat in relation.findall("./positioning/term"):
            unit_ids.append(feat.attrib["id"])

        if (unit_ids[0] in json_mentions.keys()) and (unit_ids[1] in json_mentions.keys()):
            if (len(unit_ids) == 2):
                data_character["LEFT_UNIT"] = {
                    "ID": unit_ids[0], **json_mentions[unit_ids[0]]}
                data_character["RIGHT_UNIT"] = {
                    "ID": unit_ids[1], **json_mentions[unit_ids[1]]}

            json_relations[relation.attrib["id"]] = data_character

    return(json_relations)


# return chains in json format based on the json mentions and relations.
def generate_json_chains(coreference_type=["DIRECTE", "INDIRECTE", "ANAPHORE"]):
    G = nx.Graph()
    chain_id = 0
    json_coreference_chains = {}

    for chain in json_relations:
        if json_relations[chain]["TYPE"].strip() in coreference_type:
            chain_left = json_relations[chain]["LEFT_UNIT"]["ID"]
            chain_right = json_relations[chain]["RIGHT_UNIT"]["ID"]
            G.add_edge(chain_left, chain_right,
                       coref_type=json_relations[chain]["TYPE"].strip())

    for subgraph in list(nx.connected_components(G)):
        json_coreference_chains[chain_id] = []
        for node in subgraph:
            json_coreference_chains[chain_id].append(node)
        chain_id += 1

    return(json_coreference_chains)

In [39]:
json_mentions = {}
counter=0
for unit in root_xml.iter("unit"):
    data_character = {}
    data_character["TYPE"] = unit.find("./characterisation/type").text
    counter+=1
    print(counter,len(list(root_xml.iter("unit"))))

    for anchor in root_xml.iter("anchor"):
        if anchor.attrib["id"] == unit.attrib["id"]:
            data_character["NUM"] = anchor.attrib["num"]

    for feat in unit.findall("./characterisation/featureSet/feature"):
        data_character[feat.attrib["name"]] = feat.text

    for unit_aa in root_aa.iter("unit"):
        if unit_aa.attrib["id"] == unit.attrib["id"]:
            data_character["START_ID"] = int(unit_aa.find(
                "./positioning/start/singlePosition").attrib["index"])
            data_character["END_ID"] = int(unit_aa.find(
                "./positioning/end/singlePosition").attrib["index"])

    json_mentions[unit.attrib["id"]] = data_character

1 2819
2 2819
3 2819
4 2819
5 2819
6 2819
7 2819
8 2819
9 2819
10 2819
11 2819
12 2819
13 2819
14 2819
15 2819
16 2819
17 2819
18 2819
19 2819
20 2819
21 2819
22 2819
23 2819
24 2819
25 2819
26 2819
27 2819
28 2819
29 2819
30 2819
31 2819
32 2819
33 2819
34 2819
35 2819
36 2819
37 2819
38 2819
39 2819
40 2819
41 2819
42 2819
43 2819
44 2819
45 2819
46 2819
47 2819
48 2819
49 2819
50 2819
51 2819
52 2819
53 2819
54 2819
55 2819
56 2819
57 2819
58 2819
59 2819
60 2819
61 2819
62 2819
63 2819
64 2819
65 2819
66 2819
67 2819
68 2819
69 2819
70 2819
71 2819
72 2819
73 2819
74 2819
75 2819
76 2819
77 2819
78 2819
79 2819
80 2819
81 2819
82 2819
83 2819
84 2819
85 2819
86 2819
87 2819
88 2819
89 2819
90 2819
91 2819
92 2819
93 2819
94 2819
95 2819
96 2819
97 2819
98 2819
99 2819
100 2819
101 2819
102 2819
103 2819
104 2819
105 2819
106 2819
107 2819
108 2819
109 2819
110 2819
111 2819
112 2819
113 2819
114 2819
115 2819
116 2819
117 2819
118 2819
119 2819
120 2819
121 2819
122 2819
123 2819
1

925 2819
926 2819
927 2819
928 2819
929 2819
930 2819
931 2819
932 2819
933 2819
934 2819
935 2819
936 2819
937 2819
938 2819
939 2819
940 2819
941 2819
942 2819
943 2819
944 2819
945 2819
946 2819
947 2819
948 2819
949 2819
950 2819
951 2819
952 2819
953 2819
954 2819
955 2819
956 2819
957 2819
958 2819
959 2819
960 2819
961 2819
962 2819
963 2819
964 2819
965 2819
966 2819
967 2819
968 2819
969 2819
970 2819
971 2819
972 2819
973 2819
974 2819
975 2819
976 2819
977 2819
978 2819
979 2819
980 2819
981 2819
982 2819
983 2819
984 2819
985 2819
986 2819
987 2819
988 2819
989 2819
990 2819
991 2819
992 2819
993 2819
994 2819
995 2819
996 2819
997 2819
998 2819
999 2819
1000 2819
1001 2819
1002 2819
1003 2819
1004 2819
1005 2819
1006 2819
1007 2819
1008 2819
1009 2819
1010 2819
1011 2819
1012 2819
1013 2819
1014 2819
1015 2819
1016 2819
1017 2819
1018 2819
1019 2819
1020 2819
1021 2819
1022 2819
1023 2819
1024 2819
1025 2819
1026 2819
1027 2819
1028 2819
1029 2819
1030 2819
1031 2819
1032 

1753 2819
1754 2819
1755 2819
1756 2819
1757 2819
1758 2819
1759 2819
1760 2819
1761 2819
1762 2819
1763 2819
1764 2819
1765 2819
1766 2819
1767 2819
1768 2819
1769 2819
1770 2819
1771 2819
1772 2819
1773 2819
1774 2819
1775 2819
1776 2819
1777 2819
1778 2819
1779 2819
1780 2819
1781 2819
1782 2819
1783 2819
1784 2819
1785 2819
1786 2819
1787 2819
1788 2819
1789 2819
1790 2819
1791 2819
1792 2819
1793 2819
1794 2819
1795 2819
1796 2819
1797 2819
1798 2819
1799 2819
1800 2819
1801 2819
1802 2819
1803 2819
1804 2819
1805 2819
1806 2819
1807 2819
1808 2819
1809 2819
1810 2819
1811 2819
1812 2819
1813 2819
1814 2819
1815 2819
1816 2819
1817 2819
1818 2819
1819 2819
1820 2819
1821 2819
1822 2819
1823 2819
1824 2819
1825 2819
1826 2819
1827 2819
1828 2819
1829 2819
1830 2819
1831 2819
1832 2819
1833 2819
1834 2819
1835 2819
1836 2819
1837 2819
1838 2819
1839 2819
1840 2819
1841 2819
1842 2819
1843 2819
1844 2819
1845 2819
1846 2819
1847 2819
1848 2819
1849 2819
1850 2819
1851 2819
1852 2819


2573 2819
2574 2819
2575 2819
2576 2819
2577 2819
2578 2819
2579 2819
2580 2819
2581 2819
2582 2819
2583 2819
2584 2819
2585 2819
2586 2819
2587 2819
2588 2819
2589 2819
2590 2819
2591 2819
2592 2819
2593 2819
2594 2819
2595 2819
2596 2819
2597 2819
2598 2819
2599 2819
2600 2819
2601 2819
2602 2819
2603 2819
2604 2819
2605 2819
2606 2819
2607 2819
2608 2819
2609 2819
2610 2819
2611 2819
2612 2819
2613 2819
2614 2819
2615 2819
2616 2819
2617 2819
2618 2819
2619 2819
2620 2819
2621 2819
2622 2819
2623 2819
2624 2819
2625 2819
2626 2819
2627 2819
2628 2819
2629 2819
2630 2819
2631 2819
2632 2819
2633 2819
2634 2819
2635 2819
2636 2819
2637 2819
2638 2819
2639 2819
2640 2819
2641 2819
2642 2819
2643 2819
2644 2819
2645 2819
2646 2819
2647 2819
2648 2819
2649 2819
2650 2819
2651 2819
2652 2819
2653 2819
2654 2819
2655 2819
2656 2819
2657 2819
2658 2819
2659 2819
2660 2819
2661 2819
2662 2819
2663 2819
2664 2819
2665 2819
2666 2819
2667 2819
2668 2819
2669 2819
2670 2819
2671 2819
2672 2819


In [35]:
sub_corpus
subfile

'CO2_ESLO_001_C'

# Dataset Splitting

In [46]:
# Two ways of Training: (1) commented codes down with parameter.
#                       (2) uncommented with mentioned explicity the config file.

from french_crs.fast_model_training import dataset_splitter

# ds1=dataset_splitter("./Dataset_ANCOR_Balanced.xlsx",
#                     "./Dataset_ANCOR_Balanced_Train.xlsx",
#                     "./Dataset_ANCOR_Balanced_Test.xlsx",
#                       split_config_json="./split_config.json")

# ds1.dataset_splitter_by_file(lower_rate=0.20,upper_rate=0.50, files_num=68)


ds2=dataset_splitter("./balanced_Full_Soon.xlsx.xlsx",
                    "./balanced_Full_Soon.xlsx_Train.xlsx",
                    "./balanced_Full_Soon.xlsx_Test.xlsx",
                      split_config_json="./default_split_config.json")

dict_files=ds2.dataset_splitter_by_json_config()


Created with 0.20627031160389983 rate


# Model Training

In [47]:
from french_crs.fast_model_training import model_trainer

"""
12 possibilities for "train" and "test" variables

balanced_ANAPHORE
balanced_DIRECTE
balanced_Full
balanced_INDIRECTE
representative_ANAPHORE
representative_DIRECTE
representative_Full
representative_INDIRECTE
window_ANAPHORE
window_DIRECTE
window_Full
window_INDIRECTE
"""

train="balanced_Full_Soon"
test="balanced_Full_Soon"

model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                     "../datasets/"+ test+"/"+test+"_Test.xlsx",
                     "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                     "IS_CO_REF",
                     "IS_CO_REF"
                   )

model.columns_drop_list = ["SUB_CORPUS",
                            "FILE_NAME",
                            "COREF_TABLE_ID",
                            "COREF_TYPE",
                            "COREF_ANCOR_ID",
                            "MENTION_LEFT_ID",
                            "MENTION_RIGHT_ID",
                            "LEFT_CONTENT",
                            "RIGHT_CONTENT",
                            "LEFT_DEF",
                            "RIGHT_DEF",
                            "ID_DEF"]
#                             "DISTANCE_MENTION",
#                             "DISTANCE_WORD",
#                             "DISTANCE_CHAR"]

model.convert_columns_to_numeric()
print("Model_Name :","Model_ANCOR_"+train+".model\n")
print("Train Dataset :",train+"_Train.xlsx")
print("Test  Dataset :",test+"_Test.xlsx")
performance=model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)
performance

Model_Name : Model_ANCOR_balanced_Full_Soon.model

Train Dataset : balanced_Full_Soon_Train.xlsx
Test  Dataset : balanced_Full_Soon_Test.xlsx


{'precision_score_pos': 0.8848433530906011,
 'recall_score_pos': 0.8870967741935484,
 'f1_score_pos': 0.8859686307757524,
 'precision_score_neg': 0.8638689866939611,
 'recall_score_neg': 0.8612244897959184,
 'f1_score_neg': 0.8625447112927952}

In [48]:
train="balanced_Full_Soon"
test="balanced_Full_Soon"

# SCORCH Chains Building

In [49]:
from french_crs.fast_pairs2chains import chains_builder

model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                            path_model_file="../datasets/"+ test+"/"+test+"_Test_Pred.xlsx",
                            gold_column="IS_CO_REF",
                            model_column="Prediction",
                            scorch_output_path="../",
                            threshold=0.5)

model_chains.generate_gold_model_json_output(mode="train")

SCORCH json formats saved at following addresses:
/Users/mehdi.mirzapour/French-CRS/coref_chains_gold.json
/Users/mehdi.mirzapour/French-CRS/coref_chains_pred.json


# SCORCH Outcome

In [12]:
import os
bashCommand = "scorch ./coref_chains_gold_b.json ./coref_chains_pred_r.json > ./mm.txt"
os.system(bashCommand)
f = open("./mm.txt",'r')
message = f.read()
print(message)

# Putting All Chains Together

In [ ]:
import docx
import os
from french_crs.model_training import model_trainer
from french_crs.pairs2chains import chains_builder

mydoc = docx.Document()

style = mydoc.styles['Normal']
font = style.font
font.name = 'MS Gothic'
font.size = docx.shared.Pt(10)


train_test_list=[
    "balanced_ANAPHORE",
    "balanced_DIRECTE",
    "balanced_Full",
    "balanced_INDIRECTE",
    "representative_ANAPHORE",
    "representative_DIRECTE",
    "representative_Full",
    "representative_INDIRECTE",
    "window_ANAPHORE",
    "window_DIRECTE",
    "window_Full",
    "window_INDIRECTE"
    ]

# train_test_list=[
#     ["balanced_ANAPHORE","window_ANAPHORE"],
#     ["balanced_DIRECTE","window_DIRECTE"],
#     ["balanced_Full","window_Full"],
#     ["balanced_INDIRECTE","window_INDIRECTE"],
#     ["representative_ANAPHORE","window_ANAPHORE"],
#     ["representative_DIRECTE","window_DIRECTE"],
#     ["representative_Full","window_Full"],
#     ["representative_INDIRECTE","window_INDIRECTE"],
#     ["window_ANAPHORE","window_ANAPHORE"],
#     ["window_DIRECTE","window_DIRECTE"],
#     ["window_Full","window_Full"],
#     ["window_INDIRECTE","window_INDIRECTE"]
#     ]

counter=0

# for train in train_test_list:
#     for test in train_test_list:

for train in train_test_list:
    
    test=train
    
    counter+=1

    model=model_trainer( "../datasets/"+ train+"/"+train+"_Train.xlsx",
                 "../datasets/"+ test+"/"+test+"_Test.xlsx",
                 "../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                 "IS_CO_REF",
                 "IS_CO_REF"
               )

    model.columns_drop_list = ['m1_DEF', 'm2_DEF', 'ID_DEF',
                               'DISTANCE_MENTION','DISTANCE_WORD',
                               'DISTANCE_CHAR']

    model.convert_columns_to_numeric()
    performance=model.train_model_random_forest(model_name="../pre-trained language models/Model_ANCOR_"+train+".model",max_depth=10, random_state=0,n_estimators=250)


    model_chains=chains_builder(path_gold_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx", 
                        path_model_file="../datasets/"+ train+"/"+train+"_Test_Pred.xlsx",
                        gold_column="IS_CO_REF",
                        model_column="Prediction",
                        scorch_output_path="../",
                        threshold=0.5)

    model_chains.generate_gold_model_json_output(mode="train")


    bashCommand = "scorch ../coref_chains_gold.json ../coref_chains_pred.json > ../output.txt"
    os.system(bashCommand)
    f = open("../output.txt",'r')
    message = f.read()
    f.close()


    mydoc.add_paragraph("Model_Name : "+"Model_ANCOR_"+train+".model")
    mydoc.add_paragraph("Train Dataset : "+train+"_Train.xlsx")
    mydoc.add_paragraph("Test  Dataset : "+test+"_Test.xlsx")
    mydoc.add_paragraph("\n")
    mydoc.add_paragraph(str(performance))
    mydoc.add_paragraph("\n")
    mydoc.add_paragraph(message)
    mydoc.add_page_break()

    print(counter)


mydoc.save("../pre-trained language models/Performance Analysis.docx")

#  Model Testing

In [ ]:
from model_training import model_tester

model_parameter={
                "model_name" : "./Models/Random_Forest_(Normal)_OTG_Neg_90_Pos_10.model",
                "input_file" : "./Datasets/corpus_ALL_Window_30_Test.xlsx",
                "output_file" : "./Datasets/corpus_ALL_Window_30_Test_Called_Seperately.xlsx",
                "column_gold" : "IS_CO_REF",
                "column_outcome" : "Prediction",
                "threshold" : 0.5
                }

model=model_tester(**model_parameter)
model.apply_model_to_dataset()